In [1]:
class args:
    def __init__(self):
        self.dataset = 'ml-100k'
        self.epoch = 50
        self.worker = 8
        self.verbose = 2
        self.group = 10
        self.layer = [64, 32]
        self.learn = 'retrain'
        self.deltype = 'random'
        self.delper = 5
        self.model = 'wmf'

args = args()

In [2]:
args.dataset = 'ml-100k'
args.learn = 'retrain'
args.del_type = 'random'
args.del_per = 5
args.model = 'wmf'

In [3]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath('.')))
sys.path.append(os.path.dirname(os.path.abspath('.')))
import torch
import torch.nn.functional as F
import numpy as np
from scipy.linalg import orthogonal_procrustes
import numpy as np
import torch    
import pandas as pd
from utils import seed_all, baseTrain, baseTest
from utils import WMF, DMF, GMF, NMF, BPR, hit, ndcg
from read import RatingData, PairData
from read import loadData, readRating_full, readRating_group
from config import InsParam, Instance
from scratch import Scratch

In [4]:
class CustomInstance(Instance):
    def __init__(self, param, custom_param=None):
        super().__init__(param)
        self.custom_param = custom_param
        
    def only_test(self, path=''):
        self.train_rating, self.test_rating, self.active_rating, self.inactive_rating = self.read()
        if self.param.model in ['wmf', 'dmf', 'gmf', 'nmf']:
            self.train_data = loadData(RatingData(self.train_rating), self.param.batch,
                                      self.param.n_worker,
                                      True)

        elif self.param.model in ['bpr', 'lightgcn']:
            self.train_data = loadData(PairData(self.train_rating, self.param.pos_data), self.param.batch,
                                      self.param.n_worker,
                                      True)

            self.test_data = loadData(RatingData(self.test_rating), len(self.test_rating[0]), self.param.n_worker, False)
            self.active_test_data = loadData(RatingData(self.active_rating), len(self.active_rating[0]), self.param.n_worker,
                                        False)
            self.inactive_test_data = loadData(RatingData(self.inactive_rating), len(self.inactive_rating[0]), self.param.n_worker,
                                          False)

        self.scrate = Scratch(self.param, self.param.model)
        

    def get_model(self, path=''):
        seed_all(self.scrate.seed)
        if self.scrate.model_type == 'wmf':
            model = WMF(self.scrate.n_user, self.scrate.n_item, self.scrate.k).to(self.scrate.device)
        elif self.scrate.model_type == 'bpr':
            model = BPR(self.scrate.n_user, self.scrate.n_item, self.scrate.k).to(self.scrate.device)
        elif self.scrate.model_type == 'gmf':
            model = GMF(self.scrate.n_user, self.scrate.n_item, self.scrate.k).to(self.scrate.device)
        elif self.scrate.model_type == 'nmf':
            model = NMF(self.scrate.n_user, self.scrate.n_item, self.scrate.k, self.scrate.layers).to(self.scrate.device)
        elif self.scrate.model_type == 'dmf':
            model = DMF(self.scrate.n_user, self.scrate.n_item, self.scrate.k, self.scrate.layers).to(self.scrate.device)
            
        model.load_state_dict(torch.load(path))
        
        return model
    
param = InsParam(args.dataset, args.model, args.epoch, args.worker, args.layer, args.group, args.del_per, args.learn, args.del_type)
ins = CustomInstance(param)

In [5]:
ins.only_test()
model = ins.get_model(path='../model_params/retrain/wmf_ml-100k_random_0/model.pth')
retrain_model = ins.get_model(path='../model_params/retrain/wmf_ml-100k_random_5/model.pth')

[885, 98, 46, 434, 757, 21, 851, 881, 140, 103, 88, 777, 380, 363, 316, 640, 276, 189, 175, 25, 798, 784, 511, 888, 522, 580, 458, 871, 927, 461, 531, 919, 258, 586, 822, 145, 324, 204, 561, 393, 486, 456, 814, 781, 59, 768, 438]


/tmp/ipykernel_1139949/2315595474.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))


In [7]:
model.eval()
retrain_model.eval()

WMF(
  (user_mat): Embedding(943, 32)
  (item_mat): Embedding(1349, 32)
)

In [8]:
unlearn_list = [885, 98, 46, 434, 757, 21, 851, 881, 140, 103, 88, 777, 380, 363, 316, 640, 276, 189, 175, 25, 798, 784, 511, 888, 522, 580, 458, 871, 927, 461, 531, 919, 258, 586, 822, 145, 324, 204, 561, 393, 486, 456, 814, 781, 59, 768, 438]
print(model.user_mat.weight[unlearn_list[0]])
print(retrain_model.user_mat.weight[unlearn_list[0]])

tensor([ 0.5368, -0.3249,  0.3447,  0.3734, -0.3478, -0.1079,  0.3983, -0.3578,
        -0.1621, -0.5153,  0.2502,  0.4899,  0.2659, -0.5222,  0.7066, -0.2293,
        -0.9415,  0.0398,  0.9333, -0.2786,  0.6326,  0.4908, -0.6176, -0.3008,
         0.1051,  0.2242,  0.3318, -0.0406, -0.9538, -0.4640,  0.3426,  0.4195],
       device='cuda:0', grad_fn=<SelectBackward0>)
tensor([ 4.0978e-03, -5.0318e-03, -2.0012e-03,  1.6957e-02,  5.7419e-03,
        -1.1403e-02,  4.3032e-03, -6.8148e-03,  2.0320e-03,  7.0683e-03,
        -2.0675e-04,  4.9379e-03,  4.5500e-03, -1.2044e-02,  5.2639e-03,
         8.8330e-05, -5.6867e-03,  1.4124e-02, -9.9349e-03,  4.1977e-04,
        -2.8493e-03,  9.4761e-03,  1.3664e-02,  1.2062e-03,  4.5981e-03,
        -1.6551e-02, -7.8634e-03, -1.4990e-02, -4.9395e-03, -1.6675e-02,
        -6.6674e-03,  4.0870e-03], device='cuda:0', grad_fn=<SelectBackward0>)


In [9]:
train_df = pd.DataFrame({
    "uid": ins.train_rating[0],
    "iid": ins.train_rating[1]
})

In [10]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
interaction_matrix = torch.zeros(ins.param.n_user, ins.param.n_item, device=device)
    
user_indices = train_df['uid'].values
item_indices = train_df['iid'].values
    
if user_indices.min() == 1:
    user_indices = user_indices - 1
if item_indices.min() == 1:
    item_indices = item_indices - 1
    
interaction_matrix[user_indices, item_indices] = 1
interaction_matrix

print(interaction_matrix.size())    

torch.Size([943, 1349])


In [11]:
predict = torch.matmul(model.user_mat.weight, model.item_mat.weight.T)

In [12]:
original_interaction_matrix = interaction_matrix
delete_users = [885, 98, 46, 434, 757, 21, 851, 881, 140, 103, 88, 777, 380, 363, 316, 640, 276, 189, 175, 25, 798, 784, 511, 888, 522, 580, 458, 871, 927, 461, 531, 919, 258, 586, 822, 145, 324, 204, 561, 393, 486, 456, 814, 781, 59, 768, 438]
mask = torch.ones(len(original_interaction_matrix), dtype=torch.bool)
mask[delete_users] = False
ideal_interaction_matrix = original_interaction_matrix[mask]
predict_prob = torch.sigmoid(predict)

In [13]:
def svd_based_unlearning(original_interaction_matrix, ideal_interaction_matrix, 
                        predict_prob, delete_users, device='cuda', 
                        alpha=0.5, beta=0.3, rank_ratio=0.8):
    """
    SVD 기반 언러닝 구현
    
    Args:
        original_interaction_matrix: 원본 상호작용 행렬 (num_users, num_items)
        ideal_interaction_matrix: 이상적 상호작용 행렬 (num_users-deleted, num_items)
        predict_prob: 모델 예측 확률 (num_users, num_items)
        delete_users: 삭제할 사용자 인덱스 리스트
        alpha: 아이템 공간 보정 가중치
        beta: 특이값 보정 가중치
        rank_ratio: 사용할 rank 비율
    
    Returns:
        corrected_predictions: 보정된 예측 행렬
        unlearning_metrics: 언러닝 품질 지표
    """
    
    # GPU/CPU 설정
    device = original_interaction_matrix.device
    
    print(f"Original matrix shape: {original_interaction_matrix.shape}")
    print(f"Ideal matrix shape: {ideal_interaction_matrix.shape}")
    print(f"Prediction matrix shape: {predict_prob.shape}")
    print(f"Number of users to delete: {len(delete_users)}")
    
    # 1. 각 행렬을 numpy로 변환하여 SVD 수행
    R_original = original_interaction_matrix.cpu().float().numpy()
    R_ideal = ideal_interaction_matrix.cpu().float().numpy()
    R_pred = predict_prob.detach().cpu().float().numpy()
    
    # 2. SVD 분해
    print("Performing SVD decomposition...")
    
    # Original matrix SVD
    U1, S1, V1 = np.linalg.svd(R_original, full_matrices=False)
    
    # Ideal matrix SVD
    U2, S2, V2 = np.linalg.svd(R_ideal, full_matrices=False)
    
    # Prediction matrix SVD
    U3, S3, V3 = np.linalg.svd(R_pred, full_matrices=False)
    
    print(f"SVD shapes - Original: U1{U1.shape}, S1{S1.shape}, V1{V1.shape}")
    print(f"SVD shapes - Ideal: U2{U2.shape}, S2{S2.shape}, V2{V2.shape}")
    print(f"SVD shapes - Prediction: U3{U3.shape}, S3{S3.shape}, V3{V3.shape}")
    
    # 3. 사용할 rank 결정
    min_rank = min(len(S1), len(S2), len(S3))
    target_rank = int(min_rank * rank_ratio)
    
    print(f"Using rank: {target_rank} (from min_rank: {min_rank})")
    
    # 4. 차원 축소 및 정렬
    V1_truncated = V1[:target_rank, :]
    V2_truncated = V2[:target_rank, :]
    V3_truncated = V3[:target_rank, :]
    
    S1_truncated = S1[:target_rank]
    S2_truncated = S2[:target_rank]
    S3_truncated = S3[:target_rank]
    
    # 5. Procrustes analysis로 방향 정렬
    try:
        R_align, _ = orthogonal_procrustes(V2_truncated.T, V1_truncated.T)
        V2_aligned = (V2_truncated.T @ R_align).T
    except:
        print("Procrustes alignment failed, using direct alignment")
        V2_aligned = V2_truncated
    
    # 6. 차이 계산
    Delta_V = V2_aligned - V1_truncated
    Delta_S = S2_truncated - S1_truncated
    
    print(f"Delta_V norm: {np.linalg.norm(Delta_V):.6f}")
    print(f"Delta_S norm: {np.linalg.norm(Delta_S):.6f}")
    
    # 7. 예측 행렬 보정
    V3_corrected = V3_truncated + alpha * Delta_V
    S3_corrected = S3_truncated + beta * Delta_S
    
    # 특이값 양수 유지
    S3_corrected = np.maximum(S3_corrected, 0.01 * S3_truncated)
    
    # 8. 사용자 공간 조정
    U3_corrected = U3[:, :target_rank].copy()
    
    # 삭제된 사용자들의 임베딩을 0으로 설정
    U3_corrected[delete_users, :] = 0
    
    print(f"Zeroed embeddings for {len(delete_users)} users")
    
    # 9. 보정된 예측 행렬 재구성
    R_corrected = U3_corrected @ np.diag(S3_corrected) @ V3_corrected
    
    # 10. PyTorch tensor로 변환
    corrected_predictions = torch.tensor(R_corrected, device=device, dtype=torch.float32)
    
    # 11. 확률 범위로 클리핑 (sigmoid 출력이므로 0-1 범위)
    corrected_predictions = torch.clamp(corrected_predictions, 0.0, 1.0)
    
    # 12. 언러닝 품질 평가
    
    return corrected_predictions

In [14]:
unlearned_matrix = svd_based_unlearning(original_interaction_matrix, ideal_interaction_matrix,
                     predict_prob, delete_users, 'cuda',
                    0.5, 0.3, 1.0
                    )

Original matrix shape: torch.Size([943, 1349])
Ideal matrix shape: torch.Size([896, 1349])
Prediction matrix shape: torch.Size([943, 1349])
Number of users to delete: 47
Performing SVD decomposition...
SVD shapes - Original: U1(943, 943), S1(943,), V1(943, 1349)
SVD shapes - Ideal: U2(896, 896), S2(896,), V2(896, 1349)
SVD shapes - Prediction: U3(943, 943), S3(943,), V3(943, 1349)
Using rank: 896 (from min_rank: 896)
Delta_V norm: 0.000031
Delta_S norm: 0.000000
Zeroed embeddings for 47 users


In [15]:
unlearned_matrix.size()

torch.Size([943, 1349])

In [33]:
predict_prob[delete_users[0]]

tensor([0.1728, 0.6800, 0.0808,  ..., 0.0069, 0.0023, 0.9540], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [30]:
unlearned_matrix[delete_users[0]]

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')

In [16]:
def readRating_full(train_dir, test_dir, del_type='random', del_per=5):
    train_ratings = pd.read_csv(train_dir, sep=',')
    test_ratings = pd.read_csv(test_dir, sep=',')

    del_user = delete_users
    train_ratings = train_ratings[~train_ratings['uid'].isin(del_user)].reset_index(drop=True)
    test_ratings = test_ratings[~test_ratings['uid'].isin(del_user)].reset_index(drop=True)
    # active and inactive
    user_counts = train_ratings['uid'].value_counts()
    num_active_users = int(len(user_counts) * 5 / 100)

    active_users = user_counts.index[:num_active_users].tolist()
    inactive_users = user_counts.index[num_active_users:].tolist()
    # test set
    active_ratings = test_ratings[test_ratings['uid'].isin(active_users)].reset_index(drop=True)
    inactive_ratings = test_ratings[test_ratings['uid'].isin(inactive_users)].reset_index(drop=True)

    train_rating_lists = [train_ratings['uid'], train_ratings['iid'], train_ratings['val']]
    test_rating_lists = [test_ratings['uid'], test_ratings['iid'], test_ratings['val']]

    active_ratings = [active_ratings['uid'], active_ratings['iid'], active_ratings['val']]
    inactive_ratings = [inactive_ratings['uid'], inactive_ratings['iid'], inactive_ratings['val']]

    return train_rating_lists, test_rating_lists, active_ratings, inactive_ratings
_, test_rating, _, _ = readRating_full(ins.param.train_dir, ins.param.test_dir, del_type='random', del_per=5)
origin_test_data = loadData(RatingData(ins.test_rating), len(ins.test_rating[0]), ins.param.n_worker, False)
unlearned_test_data = loadData(RatingData(test_rating), len(test_rating[0]), ins.param.n_worker, False)

In [22]:

pos_dict = np.load(ins.param.pos_data, allow_pickle=True).item()
print(baseTest(unlearned_test_data, retrain_model, 'point-wise', device, 2, pos_dict, unlearned_matrix.size(1), 20, None, None))
# print(baseTest2(unlearned_test_data, unlearned_matrix, 'point-wise', device, 2, pos_dict, unlearned_matrix.size(1), 20, None, None))

(0.3077901046343992, 0.31673471658559144)


# m

In [18]:
def baseTest2(dataloader, matrix, loss_fn, device, verbose, pos_dict, 
             n_items, top_k=20, user_mapping=None,
             pos_mapping=None):

    full_items = [i for i in range(n_items)]

    HR = []
    NDCG = []

    for user, item, rating in dataloader:
        all_users = user.unique()
        all_users = all_users.to(device)
        user = user.to(device)
        item = item.to(device)

        for uid in all_users:
            user_id = uid.item()
            user_indices = torch.where(user == uid)
            gt_items = item[user_indices].cpu().numpy().tolist()

            neg_items = list(set(full_items) - set(pos_dict[user_id]))

            new_user = torch.tensor([user_id] * len(neg_items), dtype=torch.long).to(device)
            new_item = torch.tensor(neg_items, dtype=torch.long).to(device)
            
            
            predictions = matrix[new_user, new_item]            
            # predictions = model(new_user, new_item)
            _, indices = torch.topk(predictions, top_k)
            recommends = torch.take(new_item, indices).cpu().numpy().tolist()

            HR.append(hit(gt_items, recommends))
            NDCG.append(ndcg(gt_items, recommends))

    return np.mean(NDCG), np.mean(HR)

In [19]:
print(baseTest2(unlearned_test_data, unlearned_matrix, 'point-wise', device, 2, pos_dict, unlearned_matrix.size(1), 20, None, None))

(0.3110820619083067, 0.3213303344050097)


In [26]:
retrain_user, retrain_item = retrain_model.user_mat.weight, retrain_model.item_mat.weight
retrain_predict_matrix = torch.sigmoid(retrain_user @ retrain_item.T)

In [27]:
retrain_predict_matrix.size()

torch.Size([943, 1349])